In [1]:
import numpy as np
import os
import pandas as pd

from sqlalchemy import create_engine

from nba_betting_ai.consts import proj_paths
from nba_betting_ai.data.processing import prepare_game_data, merge_data_for_indices
from nba_betting_ai.data.storage import load_games, load_teams, load_gameflow


In [2]:
postgres_user = os.environ.get('POSTGRES_USER')
postgres_password = os.environ.get('POSTGRES_PASSWORD')
postgres_host = os.environ.get('POSTGRES_HOST')
postgres_port = os.environ.get('POSTGRES_PORT')
postgres_db = os.environ.get('POSTGRES_DB')


In [3]:
postgres_conn = f'postgresql://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}'
engine = create_engine(postgres_conn)

In [4]:
df_games_raw = load_games(engine)

In [5]:
df_teams = load_teams(engine)
df_games = load_games(engine)
df_gameflow = load_gameflow(engine, game_id = list(df_games['game_id'].unique()))

In [6]:
suspicious_team_id = [50009, 15019, 12325, 15020, 15022, 12315, 15025]
suspicious_teams = df_games[df_games['team_id'].isin(suspicious_team_id)]
suspicious_teams.team_name.unique()

array(["Ra'anana Maccabi Ra'anana", 'Adelaide 36ers', 'Flamengo Flamengo',
       'New Zealand Breakers', 'Cairns Taipans', 'Madrid Baloncesto',
       'Ratiopharm Ulm'], dtype=object)

In [7]:
mask_away = df_games['matchup'].str.contains(' vs. ')
weird_games = df_games['game_id'][~mask_away][~df_games['game_id'][~mask_away].isin(df_games['game_id'][mask_away])].values
df_gameflow[df_gameflow['game_id'].isin(weird_games)].groupby('game_id').tail(1).sort_values('game_id')

,game_id,home_score,away_score,time_remaining
402156,0012400001,107,103,0
421333,0012400005,104,130,0
364342,0012400007,98,125,0
411275,0022400147,118,98,0
365083,0022401229,102,110,0
360895,0022401230,96,111,0
422643,0062400001,97,81,0


In [8]:
df_games[df_games['game_id'].isin(weird_games)].groupby('game_id').tail(1)[[ 'game_id', 'team_name','matchup', 'wl']].sort_values('game_id')

,game_id,team_name,matchup,wl
6790,0012400001,Denver Nuggets,BOS @ DEN,L
6782,0012400005,Denver Nuggets,DEN @ BOS,L
6777,0012400007,Toronto Raptors,WAS @ TOR,W
6480,0022400147,Miami Heat,MIA @ WAS,W
5896,0022401229,Atlanta Hawks,ATL @ MIL,L
5897,0022401230,Houston Rockets,HOU @ OKC,L
5867,0062400001,Milwaukee Bucks,MIL @ OKC,W


In [9]:
# from sqlalchemy import text

# with engine.connect() as conn:
#     querry = text("DELETE FROM games WHERE game_id = :game_id and wl is null")
#     conn.execute(querry, {'game_id': "0012300036"})
#     conn.commit()

In [10]:
df_games[df_games['game_id'].isin(df_games[df_games.wl.isna()]['game_id'])].game_id

Series([], Name: game_id, dtype: object)

In [11]:
len(df_games['team_id'].unique())

45

In [12]:
df_games

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl
0,2022-23,1610612743,DEN,Denver Nuggets,0042200405,2023-06-12,DEN vs. MIA,W
1,2022-23,1610612748,MIA,Miami Heat,0042200405,2023-06-12,MIA @ DEN,L
2,2022-23,1610612748,MIA,Miami Heat,0042200404,2023-06-09,MIA vs. DEN,L
3,2022-23,1610612743,DEN,Denver Nuggets,0042200404,2023-06-09,DEN @ MIA,W
4,2022-23,1610612743,DEN,Denver Nuggets,0042200403,2023-06-07,DEN @ MIA,W
...,...,...,...,...,...,...,...,...
6799,2024-25,1610612737,ATL,Atlanta Hawks,0022400506,2025-01-07,ATL @ UTA,W
6800,2024-25,1610612745,HOU,Houston Rockets,0022400503,2025-01-07,HOU @ WAS,W
6801,2024-25,1610612764,WAS,Washington Wizards,0022400503,2025-01-07,WAS vs. HOU,L
6802,2024-25,1610612747,LAL,Los Angeles Lakers,0022400504,2025-01-07,LAL @ DAL,L


In [13]:
df_games = prepare_game_data(df_games, df_gameflow)
indices = [55, 5555, 55555]
df_game_data = merge_data_for_indices(indices, df_games, df_gameflow)

In [14]:
df_game_data

,game_id,win,home_team_id,home_team_abbreviation,home_season_wins,home_season_pts_for,home_season_pts_against_so_far,home_season_games,home_last_5_wins,home_last_5_pts_for_avg,...,away_team_abbreviation,away_season_wins,away_season_pts_for,away_season_pts_against_so_far,away_season_games,away_last_5_wins,away_last_5_pts_for_avg,away_last_5_pts_for_total,away_last_5_pts_against_avg,away_last_5_pts_against_total
0,0022200412,0,1610612740,NOP,22.0,3628.0,3430.0,31,5.0,119.8,...,UTA,16.0,3806.0,3801.0,33,2.0,118.4,592.0,118.2,591.0
1,0022201126,0,1610612755,PHI,53.0,8978.0,8588.0,78,2.0,115.8,...,DEN,53.0,9155.0,8849.0,79,4.0,116.8,584.0,105.6,528.0
2,0042200405,0,1610612748,MIA,62.0,12156.0,12095.0,111,2.0,99.2,...,DEN,71.0,12180.0,11747.0,106,4.0,108.4,542.0,100.8,504.0


In [13]:
weird_games_2 = ['0032300004', '0032300005']
df_games[df_games['game_id'].isin(weird_games_2)]

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,points_for,points_against,win,season_wins,season_pts_for,season_pts_against_so_far,season_games,last_5_wins,last_5_pts_for_avg,last_5_pts_for_total,last_5_pts_against_avg,last_5_pts_against_total
6799,2023-24,1610616849,JAL,Team Jalen,0032300004,2024-02-16,JAL @ TAM,W,40,35,1,1.0,26.0,13.0,1,2.0,28.666667,86.0,23.333333,70.0
6795,2023-24,1610616848,DLF,Team Detlef,0032300005,2024-02-16,DLF @ PAU,W,41,36,1,0.0,13.0,26.0,1,0.0,19.000000,38.0,33.000000,66.0
6792,2023-24,1610616847,PAU,Team Pau,0032300005,2024-02-16,PAU vs. DLF,L,36,41,0,NaN,NaN,NaN,0,2.0,32.500000,65.0,22.500000,45.0
6801,2023-24,1610616850,TAM,Team Tamika,0032300004,2024-02-16,TAM vs. JAL,L,35,40,0,NaN,NaN,NaN,0,0.0,32.000000,32.0,40.000000,40.0


In [17]:
df_games[df_games_raw['game_id'].isin(weird_games)].sort_values(by='game_id')


/tmp/ipykernel_194216/2082755278.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_games[df_games_raw['game_id'].isin(weird_games)].sort_values(by='game_id')


,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,points_for,points_against,win,season_wins,season_pts_for,season_pts_against_so_far,season_games,last_5_wins,last_5_pts_for_avg,last_5_pts_for_total,last_5_pts_against_avg,last_5_pts_against_total
6479,2023-24,1610612765,DET,Detroit Pistons,0022300474,2024-01-03,DET @ UTA,L,148,154,0,5.0,4118.0,4500.0,37,1.0,118.2,591.0,127.0,635.0
6480,2023-24,1610612765,DET,Detroit Pistons,0022300490,2024-01-05,DET @ GSW,L,109,113,0,5.0,4266.0,4654.0,38,1.0,124.8,624.0,132.6,663.0
5866,2023-24,1610612762,UTA,Utah Jazz,0022300966,2024-03-15,UTA vs. ATL,W,124,122,1,31.0,8205.0,8390.0,70,1.0,118.4,592.0,125.0,625.0
5867,2023-24,1610612762,UTA,Utah Jazz,0022300974,2024-03-16,UTA vs. MIN,L,100,119,0,32.0,8329.0,8512.0,71,2.0,119.2,596.0,124.2,621.0
5894,2024-25,1610612762,UTA,Utah Jazz,0022400148,2024-11-02,UTA @ DEN,L,103,129,0,4.0,1164.0,1248.0,11,0.0,99.2,496.0,116.4,582.0
5895,2024-25,1610612762,UTA,Utah Jazz,0022400160,2024-11-04,UTA @ CHI,W,135,126,1,4.0,1267.0,1377.0,12,0.0,95.0,475.0,117.0,585.0
5896,2024-25,1610612762,UTA,Utah Jazz,0022400182,2024-11-07,UTA @ MIL,L,100,123,0,5.0,1402.0,1503.0,13,1.0,104.8,524.0,116.8,584.0
5897,2024-25,1610612762,UTA,Utah Jazz,0022400197,2024-11-09,UTA @ SAS,W,111,110,1,5.0,1502.0,1626.0,14,1.0,104.4,522.0,119.4,597.0
6774,2024-25,1610612766,CHA,Charlotte Hornets,0022400373,2024-12-20,CHA @ PHI,L,98,108,0,9.0,3447.0,3600.0,32,1.0,106.4,532.0,115.6,578.0
6777,2024-25,1610612766,CHA,Charlotte Hornets,0022400428,2024-12-28,CHA vs. OKC,L,94,106,0,9.0,3756.0,3935.0,35,0.0,106.2,531.0,115.8,579.0


In [18]:
df_games[df_games['game_id'].isin(weird_games)]

,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,points_for,points_against,win,season_wins,season_pts_for,season_pts_against_so_far,season_games,last_5_wins,last_5_pts_for_avg,last_5_pts_for_total,last_5_pts_against_avg,last_5_pts_against_total
451,2024-25,1610612738,BOS,Boston Celtics,0012400001,2024-10-04,BOS @ DEN,W,107,103,1,NaN,NaN,NaN,0,4.0,101.6,508.0,99.2,496.0
1598,2024-25,1610612743,DEN,Denver Nuggets,0012400001,2024-10-04,BOS @ DEN,L,103,107,0,NaN,NaN,NaN,0,3.0,100.8,504.0,101.4,507.0
1599,2024-25,1610612743,DEN,Denver Nuggets,0012400005,2024-10-06,DEN @ BOS,L,104,130,0,0.0,103.0,107.0,1,2.0,98.0,490.0,104.8,524.0
6317,2024-25,1610612764,WAS,Washington Wizards,0012400007,2024-10-06,WAS @ TOR,L,98,125,0,NaN,NaN,NaN,0,0.0,118.8,594.0,125.8,629.0
452,2024-25,1610612738,BOS,Boston Celtics,0012400005,2024-10-06,DEN @ BOS,W,130,104,1,1.0,107.0,103.0,1,4.0,101.6,508.0,102.0,510.0
5669,2024-25,1610612761,TOR,Toronto Raptors,0012400007,2024-10-06,WAS @ TOR,W,125,98,1,NaN,NaN,NaN,0,1.0,112.2,561.0,122.2,611.0
6326,2024-25,1610612764,WAS,Washington Wizards,0022400147,2024-11-02,MIA @ WAS,L,98,118,0,4.0,987.0,1081.0,9,3.0,118.0,590.0,122.6,613.0
2760,2024-25,1610612748,MIA,Miami Heat,0022400147,2024-11-02,MIA @ WAS,W,118,98,1,6.0,987.0,983.0,9,3.0,107.6,538.0,109.0,545.0
229,2024-25,1610612737,ATL,Atlanta Hawks,0022401229,2024-12-14,ATL @ MIL,L,102,110,0,15.0,3465.0,3554.0,30,4.0,119.2,596.0,117.8,589.0
5483,2024-25,1610612760,OKC,Oklahoma City Thunder,0022401230,2024-12-14,HOU @ OKC,W,111,96,1,23.0,3349.0,3002.0,29,4.0,123.0,615.0,106.0,530.0
